<a href="https://colab.research.google.com/github/granantuin/Model_vs_data_label/blob/master/all_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix,accuracy_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate,GridSearchCV
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV

In [0]:
def evaluate(y_test,y_pred):
  index=["E","N","NE","NW","S","SE","SW","W"]
  print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=index, columns=index))
  print("****************")
  print("Accuracy=","{:.2%}".format(accuracy_score(y_test, y_pred)),"// Model Accuracy=45%")
  results= precision_recall_fscore_support(y_test, y_pred, average=None, )
  df=pd.DataFrame({"Precision":results[0],"Recall":results[1],"F1":results[2],"W_DIR":index})
  df=df.set_index("W_DIR")
  print("Average precision =","{:.2%}".format(df["Precision"].mean()),"// Model precision=28%")
  print("Average recall =","{:.2%}".format(df["Recall"].mean()),"// Model recall=28%")
  results= precision_recall_fscore_support(y_test, y_pred, average="micro", )
  print("Precision micro=","{:.2%}".format(results[0]),"//Model precision micro=45%")
  print("Recall micro=","{:.2%}".format(results[1]),"//Model recall micro=45%")
  results= precision_recall_fscore_support(y_test, y_pred, average='weighted', )
  print("Precision weighted=","{:.2%}".format(results[0]),"//Model weighted=56%")
  print("Recall weighted =","{:.2%}".format(results[1]),"//Model weighted=53%")
  print("****************")
  print(df)

In [0]:

from google.colab import drive
drive.mount('/content/drive')


In [0]:
drive_4km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_4km_h24toh48_dir/"
drive_1km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/"
dr=[drive_4km,drive_1km]

In [0]:
y_data=pd.read_excel(dr[1]+"y_data.xlsx")
x_data=pd.read_csv(dr[1]+"x_data",usecols=range(1,10))

In [0]:
NE=y_data[(y_data["value"]>22.5) & (y_data["value"]<67.5)]
NE["label"]="NE"
E=y_data[(y_data["value"]>67.5) & (y_data["value"]<112.5)]
E["label"]="E"
SE=y_data[(y_data["value"]>112.5) & (y_data["value"]<157.5)]
SE["label"]="SE"
S=y_data[(y_data["value"]>157.5) & (y_data["value"]<202.5)]
S["label"]="S"
SW=y_data[(y_data["value"]>202.5) & (y_data["value"]<247.5)]
SW["label"]="SW"
W=y_data[(y_data["value"]>247.5) & (y_data["value"]<292.5)]
W["label"]="W"
NW=y_data[(y_data["value"]>292.5) & (y_data["value"]<337.5)]
NW["label"]="NW"
N=y_data[(y_data["value"]>337.5) | (y_data["value"]<22.5)]
N["label"]="N"
winds=[NE,N,E,SE,S,SW,W,NW]
y_data=pd.concat(winds)
y_data=y_data.sort_values(by="date")

**Display**

In [0]:
pd.concat([x_data,y_data],axis=1).set_index('date')

**Classifier SVM**

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
evaluate(y_test,y_pred)


    E  N  NE  NW  S  SE  SW  W
E   0  0  11   0  0   0   0  0
N   0  0  17   0  0   0   0  0
NE  0  0  97   0  0   0   0  0
NW  0  0  11   0  0   0   0  0
S   0  0  11   0  0   0   1  0
SE  0  0  11   0  0   0   0  0
SW  0  0  26   0  0   0   0  0
W   0  0  19   0  0   0   0  0
****************
Accuracy= 47.55% // Model Accuracy=45%
Average precision = 5.97% // Model precision=28%
Average recall = 12.50% // Model recall=28%
Precision micro= 47.55% //Model precision micro=45%
Recall micro= 47.55% //Model recall micro=45%
Precision weighted= 22.72% //Model weighted=56%
Recall weighted = 47.55% //Model weighted=53%
****************
       Precision  Recall        F1
W_DIR                             
E       0.000000     0.0  0.000000
N       0.000000     0.0  0.000000
NE      0.477833     1.0  0.646667
NW      0.000000     0.0  0.000000
S       0.000000     0.0  0.000000
SE      0.000000     0.0  0.000000
SW      0.000000     0.0  0.000000
W       0.000000     0.0  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Classifier Dummy**

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
y_pred = DummyClassifier(strategy='most_frequent').fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)



    E  N  NE  NW  S  SE  SW  W
E   0  0  11   0  0   0   0  0
N   0  0  17   0  0   0   0  0
NE  0  0  97   0  0   0   0  0
NW  0  0  11   0  0   0   0  0
S   0  0  12   0  0   0   0  0
SE  0  0  11   0  0   0   0  0
SW  0  0  26   0  0   0   0  0
W   0  0  19   0  0   0   0  0
****************
Accuracy= 47.55% // Model Accuracy=45%
Average precision = 5.94% // Model precision=28%
Average recall = 12.50% // Model recall=28%
Precision micro= 47.55% //Model precision micro=45%
Recall micro= 47.55% //Model recall micro=45%
Precision weighted= 22.61% //Model weighted=56%
Recall weighted = 47.55% //Model weighted=53%
****************
       Precision  Recall        F1
W_DIR                             
E        0.00000     0.0  0.000000
N        0.00000     0.0  0.000000
NE       0.47549     1.0  0.644518
NW       0.00000     0.0  0.000000
S        0.00000     0.0  0.000000
SE       0.00000     0.0  0.000000
SW       0.00000     0.0  0.000000
W        0.00000     0.0  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Classifier Randomforest**

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
y_pred = RandomForestClassifier(n_estimators=20).fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)

    E  N  NE  NW  S  SE  SW  W
E   0  0   8   0  0   0   2  1
N   0  6   7   1  0   0   0  3
NE  1  2  84   0  4   0   4  2
NW  1  0   3   0  0   0   3  4
S   0  0   4   0  0   1   4  3
SE  0  0   4   0  1   3   0  3
SW  0  0   7   1  0   0  16  2
W   0  0   6   2  0   0   2  9
****************
Accuracy= 57.84% // Model Accuracy=45%
Average precision = 37.90% // Model precision=28%
Average recall = 32.26% // Model recall=28%
Precision micro= 57.84% //Model precision micro=45%
Recall micro= 57.84% //Model recall micro=45%
Precision weighted= 52.45% //Model weighted=56%
Recall weighted = 57.84% //Model weighted=53%
****************
       Precision    Recall        F1
W_DIR                               
E       0.000000  0.000000  0.000000
N       0.750000  0.352941  0.480000
NE      0.682927  0.865979  0.763636
NW      0.000000  0.000000  0.000000
S       0.000000  0.000000  0.000000
SE      0.750000  0.272727  0.400000
SW      0.516129  0.615385  0.561404
W       0.333333  0.473684  0

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


**Oversample minority class**

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
X = pd.concat([x_train, y_train], axis=1)
not_NE = X[X["label"]!="NE"]
NE = X[X["label"]=="NE"]
not_NE_upsampled = resample(not_NE,
                          replace=True, # sample with replacement
                          n_samples=len(not_NE), # match number in majority class
                          random_state=27) # reproducible results
upsampled = pd.concat([NE, not_NE_upsampled])
y_train=upsampled["label"]
x_train=upsampled.drop("label",axis=1)
y_pred = RandomForestClassifier(n_estimators=20).fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)


**Generate synthetic samples**

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
print("Original train samples=",x_train.shape[0])
#y_pred = RandomForestClassifier(n_estimators=15).fit(x_train, y_train).predict(x_test)
#evaluate(y_test["label"],y_pred)

sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
print("Synthetic train samples=",x_train.shape[0])
print("****Randomforest****")
y_pred = RandomForestClassifier(n_estimators=20).fit(x_train, y_train).predict(x_test)
evaluate(y_test["label"],y_pred)
print("****SVM****")
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
evaluate(y_test["label"],y_pred)


Original train samples= 612
Synthetic train samples= 2288
****Randomforest****
    E  N  NE  NW  S  SE  SW  W
E   3  2   8   1  0   0   1  0
N   1  3   7   0  0   1   1  0
NE  9  3  68   2  4   3   1  0
NW  0  1   2   1  0   1   2  0
S   0  0   1   1  5   2   5  1
SE  0  0   3   2  1   1   1  1
SW  0  1   4   2  5   4  10  3
W   1  3   6   5  2   2   3  4
****************
Accuracy= 46.57% // Model Accuracy=45%
Average precision = 30.38% // Model precision=28%
Average recall = 28.40% // Model recall=28%
Precision micro= 46.57% //Model precision micro=45%
Recall micro= 46.57% //Model recall micro=45%
Precision weighted= 47.66% //Model weighted=56%
Recall weighted = 46.57% //Model weighted=53%
****************
       Precision    Recall        F1
W_DIR                               
E       0.214286  0.200000  0.206897
N       0.230769  0.230769  0.230769
NE      0.686869  0.755556  0.719577
NW      0.071429  0.142857  0.095238
S       0.294118  0.333333  0.312500
SE      0.071429  0.1111

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


    E  N  NE  NW  S  SE  SW  W
E   0  1  14   0  0   0   0  0
N   0  0  13   0  0   0   0  0
NE  0  0  90   0  0   0   0  0
NW  0  0   5   1  0   0   0  1
S   0  0  12   0  0   0   3  0
SE  0  0   9   0  0   0   0  0
SW  0  0  22   0  2   0   4  1
W   0  1  24   0  0   0   0  1
****************
Accuracy= 47.06% // Model Accuracy=45%
Average precision = 29.76% // Model precision=28%
Average recall = 16.49% // Model recall=28%
Precision micro= 47.06% //Model precision micro=45%
Recall micro= 47.06% //Model recall micro=45%
Precision weighted= 36.81% //Model weighted=56%
Recall weighted = 47.06% //Model weighted=53%
****************
       Precision    Recall        F1
W_DIR                               
E       0.000000  0.000000  0.000000
N       0.000000  0.000000  0.000000
NE      0.476190  1.000000  0.645161
NW      1.000000  0.142857  0.250000
S       0.000000  0.000000  0.000000
SE      0.000000  0.000000  0.000000
SW      0.571429  0.137931  0.222222
W       0.333333  0.038462  0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Cross validation K_folds**

In [30]:
clf=RandomForestClassifier(n_estimators=5)
scoring = ['precision_macro', 'recall_macro','f1_macro',"accuracy"]
scores = cross_validate(clf, x_data, y_data["label"], scoring=scoring,
                        cv=10, return_train_score=False)
print(scores["test_accuracy"])
print(scores["test_f1_macro"])

print("Accuracy: {:.2%} (+/- {:.2%})" .format (scores["test_accuracy"].mean(), scores["test_accuracy"].std() * 2))
print("Recall: {:.2%} (+/- {:.2%})" .format (scores["test_recall_macro"].mean(), scores["test_recall_macro"].std() * 2))
print("Precision: {:.2%} (+/- {:.2%})" .format (scores["test_precision_macro"].mean(), scores["test_precision_macro"].std() * 2))
print("f1: {:.2%} (+/-{:.2%} )".format (scores["test_f1_macro"].mean(), scores["test_f1_macro"].std() * 2))

[0.41666667 0.47619048 0.54761905 0.46987952 0.52439024 0.45679012
 0.65       0.4625     0.425      0.41025641]
[0.24100529 0.26372896 0.2793254  0.31212464 0.23127833 0.2469246
 0.38714255 0.27010346 0.27597403 0.1812181 ]
Accuracy: 48.39% (+/- 13.88%)
Recall: 27.45% (+/- 11.56%)
Precision: 30.33% (+/- 13.40%)
f1: 26.89% (+/-10.28% )


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py

**Tuning SVM**

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
tuned_parameters = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
#Generate synthetic samples
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
clf = GridSearchCV(svm.SVC(), tuned_parameters,scoring="f1_macro", cv=5).fit(x_train, y_train)
                    
print("Best parameters=",clf.best_params_)
print("Best F1=",clf.best_score_)
#SVC(C=1000, gamma=0.001, kernel='rbf')


Best parameters= {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Best F1= 0.8435515015134369


**Evaluate SVM with best parameters**

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
#Generate synthetic samples
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
y_pred=svm.SVC(C=100, gamma=0.001, kernel='rbf').fit(x_train,y_train).predict(x_test)
evaluate(y_test["label"],y_pred)

    E  N  NE  NW  S  SE  SW  W
E   1  3  10   0  0   0   1  0
N   0  3   8   1  0   0   0  1
NE  3  6  74   0  2   1   3  1
NW  0  1   3   1  0   0   1  1
S   0  0   4   1  4   2   3  1
SE  1  0   4   1  1   0   1  1
SW  2  1   5   3  7   3   5  3
W   0  5   9   2  4   1   3  2
****************
Accuracy= 44.12%  Model Accuracy=32.17%
****************
Average precision = 22.01%  Model precision=21.03%
Average recall = 22.23%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.142857  0.066667  0.090909
N       0.157895  0.230769  0.187500
NE      0.632479  0.822222  0.714976
NW      0.111111  0.142857  0.125000
S       0.222222  0.266667  0.242424
SE      0.000000  0.000000  0.000000
SW      0.294118  0.172414  0.217391
W       0.200000  0.076923  0.111111


**Tuning RandomForestClassifier**

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
# Synthetic samples
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
rf_random.fit(x_train, y_train)
print("best parameters",rf_random.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.8min finished


best parameters {'n_estimators': 800, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}


**Running Randomforest tuned**

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=27)
#Generate synthetic samples
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
clas=RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=70, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
y_pred = clas.fit(x_train, y_train).predict(x_test)
evaluate(y_test["label"],y_pred)

    E   N  NE  NW  S  SE  SW   W
E   2   3   8   1  3   0   1   1
N   0   7   8   1  0   0   1   0
NE  4  11  79   3  4   3   0   1
NW  0   1   2   2  1   0   2   0
S   0   1   2   2  7   2   4   1
SE  0   1   5   0  1   0   1   1
SW  1   1   5   3  6   6   8   7
W   1   2   7   2  5   2   2  10
****************
Accuracy= 46.94%  Model Accuracy=32.17%
****************
Average precision = 31.12%  Model precision=21.03%
Average recall = 30.33%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.250000  0.105263  0.148148
N       0.259259  0.411765  0.318182
NE      0.681034  0.752381  0.714932
NW      0.142857  0.250000  0.181818
S       0.259259  0.368421  0.304348
SE      0.000000  0.000000  0.000000
SW      0.421053  0.216216  0.285714
W       0.476190  0.322581  0.384615


**Cross validation K_folds**

In [0]:
scoring = ['precision_macro', 'recall_macro','f1_macro',"accuracy"]
scores = cross_validate(clas, x_data, y_data["label"], scoring=scoring,
                        cv=20, return_train_score=False)
print("Accuracy: {:.2%} (+/- {:.2%})" .format (scores["test_accuracy"].mean(), scores["test_accuracy"].std() * 2))
print("Recall: {:.2%} (+/- {:.2%})" .format (scores["test_recall_macro"].mean(), scores["test_recall_macro"].std() * 2))
print("Precision: {:.2%} (+/- {:.2%})" .format (scores["test_precision_macro"].mean(), scores["test_precision_macro"].std() * 2))
print("f1: {:.2%} (+/-{:.2%} )".format (scores["test_f1_macro"].mean(), scores["test_f1_macro"].std() * 2))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py

Accuracy: 33.58% (+/- 33.58%)
Recall: 28.44% (+/- 28.08%)
Precision: 20.75% (+/- 25.29%)
f1: 22.75% (+/-26.01% )


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py